Imports needed packages

In [1]:
from fastai.collab import *
from fastai.tabular.all import *

Creates Class for DotProductBias and T

In [35]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        movies = self.movie_factors[x[:,1]]
        res = (users*movies).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)
    
class T(Module):
    def __init__(self): self.a = nn.Parameter(torch.ones(3))
    def __init__(self): self.a = nn.Linear(1,3, bias=False)
    

Reads data from ratings.csv into "ratings" object

In [3]:
ratings = pd.read_csv(r"C:\Users\dfent\Desktop\ml-25m\ratings.csv", delimiter=',', header=0)

Reads data from movies.csv into "movies" object

In [10]:
movies = pd.read_csv(r"C:\Users\dfent\Desktop\ml-25m\movies.csv", delimiter=',', header=0)

Combines movies and ratings objects on movie id

In [12]:
ratings = ratings.merge(movies)

In [13]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


Creates dataloader from ratings object

In [14]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=400)
dls.show_batch()

,userId,title,rating
0,141138,Léon: The Professional (a.k.a. The Professional) (Léon) (1994),5.0
1,141814,Less Than Zero (1987),3.0
2,118909,When Harry Met Sally... (1989),4.0
3,113127,Jumper (2008),4.5
4,45203,Star Wars: Episode VII - The Force Awakens (2015),5.0
5,2020,Dead Man Walking (1995),3.5
6,122663,Blues Brothers 2000 (1998),3.0
7,157975,Sleep Tight (Mientras duermes) (2011),3.5
8,54353,My Crazy Life (Mi vida loca) (1993),3.0
9,51760,Tangled (2010),3.5


In [15]:
n_users = len(dls.classes['userId'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [16]:
one_hot_3 = one_hot(3, n_users).float()
user_factors[3]


tensor([ 0.0762, -0.5269, -0.3519,  0.5022,  0.4439])

In [17]:
x,y = dls.one_batch()
x.shape

torch.Size([400, 2])

In [36]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.817340,0.825204,07:02
1,0.807076,0.821226,07:07
2,0.777674,0.789576,06:58
3,0.737186,0.741160,06:55
4,0.708877,0.715677,06:55


In [29]:
L(T().parameters())

(#1) [Parameter containing:
tensor([[-0.4460],
        [ 0.8897],
        [-0.5261]], requires_grad=True)]

In [30]:
t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.2201],
        [ 0.7695],
        [ 0.6111]], requires_grad=True)]

In [31]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [32]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [38]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:10]
[dls.classes['title'][i] for i in idxs]

['Battlefield Earth (2000)',
 'Epic Movie (2007)',
 'Glitter (2001)',
 'Gigli (2003)',
 'Son of the Mask (2005)',
 'Disaster Movie (2008)',
 'Turbo: A Power Rangers Movie (1997)',
 'Home Alone 3 (1997)',
 'Dumb and Dumberer: When Harry Met Lloyd (2003)',
 'Spice World (1997)']

In [40]:
learn.export()

In [42]:
idxs = movie_bias.argsort(descending=True)[:20]
[dls.classes['title'][i]for i in idxs]

['Shawshank Redemption, The (1994)',
 'Usual Suspects, The (1995)',
 "Schindler's List (1993)",
 'Godfather, The (1972)',
 'Silence of the Lambs, The (1991)',
 'Fight Club (1999)',
 'Pulp Fiction (1994)',
 'Forrest Gump (1994)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Godfather: Part II, The (1974)',
 'Fugitive, The (1993)',
 'Matrix, The (1999)',
 '12 Angry Men (1957)',
 'Life Is Beautiful (La Vita è bella) (1997)',
 'American History X (1998)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Inception (2010)',
 'Planet Earth (2006)',
 'Princess Bride, The (1987)',
 'Dark Knight, The (2008)']